In [1]:
import pandas as pd
import numpy as np

In [10]:
al = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Aluminum/latest/PCU3313153313150.csv", parse_dates  = ['DATE'])
al_sheet_strip = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Commodity_Metals_Aluminum_Sheet_Strip/latest/WPU10250105.csv", parse_dates  = ['DATE'])
cu_br_mill_shape = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Copper_Brass_Mill_Shapes/latest/WPU102502.csv", parse_dates  = ['DATE'])
bauxite = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Industry_Alumina_Aluminum_Bauxite/latest/PCU3313133131.csv", parse_dates  = ['DATE'])
iron_ore = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Industry_Iron_Ore_Mining/latest/PCU2122121221.csv", parse_dates  = ['DATE'])
plastic = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Industry_Plastics_Products_Manufacturing/latest/PCU32613261.csv", parse_dates  = ['DATE'])
iron_steel = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Metals_Iron_Steel/latest/WPU101.csv", parse_dates  = ['DATE'])
stainless_steel = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Metals_Steel_Wire_Stainless_Steel/latest/WPU10170502.csv", parse_dates  = ['DATE'])
nikel = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Nickel/latest/WPU102504.csv", parse_dates  = ['DATE'])
zinc = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Nonferrous_Scrap_Lead_Zinc_Precious_Metals/latest/WPU10230309.csv", parse_dates  = ['DATE'])
rubber_plastic = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Rubber_Plastic_Products/latest/WPU07.csv", parse_dates  = ['DATE'])
cu = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Special_Index_Copper/latest/WPUSI019011.csv", parse_dates  = ['DATE'])
nylon = pd.read_csv("s3://landing-external/rearc-fred-commodities/fred/PPI_Textile_Products_Nylon/latest/WPU031502412.csv", parse_dates  = ['DATE'])

In [16]:
al.sort_values(by = "DATE", ascending = False).head()[:2].reset_index()

,index,DATE,PCU3313153313150
0,484,2021-10-01,244.0
1,483,2021-09-01,245.3


In [57]:
dfs = []
for df in [al, al_sheet_strip, cu_br_mill_shape, bauxite, iron_ore, plastic, iron_steel, stainless_steel, nikel, zinc, rubber_plastic,
          cu, nylon]:
    temp_df = df.sort_values(by = "DATE", ascending = True).tail(2)
    temp_df['Base Commodity'] = temp_df.columns[-1]
    temp_df.columns = ['DATE', 'Base Commodity PPI', 'Base Commodity']
    temp_df['Base Commodity PPI'] = temp_df['Base Commodity PPI'].astype(float)
    temp_df['Base Commodity Change%'] = temp_df['Base Commodity PPI'].pct_change()*100
    dfs.append(temp_df)

In [60]:
final_df = pd.concat(dfs, axis = 0).reset_index(drop = True).dropna()
final_df["Base Commodity PPI Change"] = ['Increase' if i > 0 else "Decrease" for i in final_df['Base Commodity Change%']]

In [61]:
final_df.head()

,DATE,Base Commodity PPI,Base Commodity,Base Commodity Change%,Base Commodity PPI Change
1,2021-10-01,244.0,PCU3313153313150,-0.529963,Decrease
3,2021-10-01,166.7,WPU10250105,-0.418160,Decrease
5,2021-10-01,598.0,WPU102502,-0.033434,Decrease
7,2021-10-01,171.8,PCU3313133131,1.958457,Increase
9,2021-10-01,187.6,PCU2122121221,-1.263158,Decrease


In [62]:
final_df['Base Commodity'].unique()

array(['PCU3313153313150', 'WPU10250105', 'WPU102502', 'PCU3313133131',
       'PCU2122121221', 'PCU32613261', 'WPU101', 'WPU10170502',
       'WPU102504', 'WPU10230309', 'WPU07', 'WPUSI019011', 'WPU031502412'],
      dtype=object)

In [63]:
commodity_map = {"WPU10170502": 'Stainless steel', 'WPU101': 'Steel', 'PCU32613261': 'Plastic', 'WPU031502412': 'Nylon',
                'PCU3313133131': 'Aluminum', 'WPUSI019011': 'Copper', 'WPU07': 'Rubber', 'WPU102502': 'Brass',
                'WPU10230309': 'Zinc die-casting'}
final_df['Base Commodity'] = final_df['Base Commodity'].replace(commodity_map)

In [64]:
final_df.head()

,DATE,Base Commodity PPI,Base Commodity,Base Commodity Change%,Base Commodity PPI Change
1,2021-10-01,244.0,PCU3313153313150,-0.529963,Decrease
3,2021-10-01,166.7,WPU10250105,-0.418160,Decrease
5,2021-10-01,598.0,Brass,-0.033434,Decrease
7,2021-10-01,171.8,Aluminum,1.958457,Increase
9,2021-10-01,187.6,PCU2122121221,-1.263158,Decrease


In [65]:
final_df.to_csv("CommodityPPI.csv", index = False)